In [3]:
# to be used with runpod/pytorch:2.1.0-py3.10-cuda11.8.0-devel-ubuntu22.04

In [4]:
# Each chekcpoint in this notebook is 130 mb, old are not deleted when new are here, calculate required storage as well

In [1]:
print("Ready!")

Ready!


In [7]:
# Installing stuff, onl

In [ ]:
pip uninstall -y unsloth unsloth_zoo torch xformers transformers triton protobuf wheel peft trl accelerate

In [ ]:
# First: install correct torch
!pip install torch==2.6.0 torchvision

# Then: install remaining tools with pinned versions
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo

# Fix any known dependency mismatches
!pip install \
    "protobuf<4.0.0" \
    "wheel>=0.42.0" \
    sentencepiece \
    "datasets>=3.4.1" \
    huggingface_hub \
    hf_transfer \
    transformers==4.51.3 \
    safetensors \
    msgspec \
    tyro \
    regex \
    rich

# Finally: install Unsloth itself
!pip install --no-deps unsloth

In [1]:
#Loading Unsloth Models
from unsloth import FastModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!


In [2]:
# Importing other required models
from datasets import load_dataset
from datasets import Dataset
import json

In [ ]:
# Logging into hugigngface as some of our datasets are gated!
from huggingface_hub import login
readonly_token = "<enterhere>"
read_and_write_token = "<enterhere>"
login(token=read_and_write_token)

In [4]:
# Preparing our model
max_seq_length_ = 4096

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = max_seq_length_, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.6.1: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [5]:
# Loading model base
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [6]:
# Dataset Download from huggingface
# 1. CodeAnalyst
python_dataset = load_dataset("flytech/python-codes-25k")
# 2. Text-to-SQL
sql_dataset = load_dataset("gretelai/synthetic_text_to_sql")
# 3. PySpark QA
pyspark_dataset = load_dataset("irfanalisoomro/clean_generated_pyspark_answers")
# 4. Leetcode
# leetcode_dataset = load_dataset("DenCT/leetcode-python-solutions-with-exaplanations")
leetcode_dataset = load_dataset("LimYeri/LeetCode_Python_Solutions_v2")
# 5. Neetcode
neetcode_dataset = load_dataset("nischalon10/neetcode")

README.md:   0%|          | 0.00/3.29k [00:00<?, ?B/s]

python-codes-25k.json:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

python-codes-25k.jsonl:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49626 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/709k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/469k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1990 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/149 [00:00<?, ? examples/s]

In [7]:
# Sample conversion function for each dataset type
def convert_to_gemma_format(user_text, model_text):
    return f"<start_of_turn>user\n{user_text}<end_of_turn>\n<start_of_turn>model\n{model_text}<end_of_turn>"

# --- Load your real datasets below instead of these small examples ---

def format_python_dataset(example):
    prompt = f"{example['instruction']}\n{example['input']}".strip()
    return {"text": convert_to_gemma_format(prompt, example["output"])}

# 2. Text-to-SQL dataset
def format_sql_dataset(example):
    prompt = f"{example['sql_prompt']}\n{example['sql_context']}".strip()
    model_reply = f"{example['sql']}\n\nExplanation:\n{example['sql_explanation']}"
    return {"text": convert_to_gemma_format(prompt, model_reply)}

# 3. PySpark Q&A dataset
def format_pyspark_dataset(example):
    return {"text": convert_to_gemma_format(example["Question"], example["Answer"])}

# 4. Leetcode solutions dataset
# def format_leetcode_dataset(example):
#     return {"text": convert_to_gemma_format(example["question_content"], example["content"])}

def format_leetcode_dataset(example):
    return {"text": convert_to_gemma_format(example["question_content"], example["content"])}

def format_neetcode_dataset(example):
    return {"text": convert_to_gemma_format(example["content"], example["python"])}

In [8]:
# Map your formatting functions on each dataset's "train" split, removing original columns.
formatted_python = python_dataset["train"].map(
    format_python_dataset, 
    remove_columns=python_dataset["train"].column_names
)
formatted_sql = sql_dataset["train"].map(
    format_sql_dataset, 
    remove_columns=sql_dataset["train"].column_names
)
formatted_pyspark = pyspark_dataset["train"].map(
    format_pyspark_dataset, 
    remove_columns=pyspark_dataset["train"].column_names
)
formatted_leetcode = leetcode_dataset["train"].map(
    format_leetcode_dataset, 
    remove_columns=leetcode_dataset["train"].column_names
)
formatted_neetcode = neetcode_dataset["train"].map(
    format_neetcode_dataset, 
    remove_columns=neetcode_dataset["train"].column_names
)

Map:   0%|          | 0/49626 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8886 [00:00<?, ? examples/s]

Map:   0%|          | 0/15734 [00:00<?, ? examples/s]

Map:   0%|          | 0/1990 [00:00<?, ? examples/s]

In [9]:
# Combine all formatted results into one Dataset containing only the "text" field.
combined_texts = (
    formatted_python["text"] +
    formatted_sql["text"] +
    formatted_pyspark["text"] +
    formatted_leetcode["text"] +
    formatted_neetcode["text"]
)
gemma_dataset = Dataset.from_dict({"text": combined_texts})

In [10]:
gemma_dataset

Dataset({
    features: ['text'],
    num_rows: 176236
})

In [11]:
# Training the model with custom settings
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = gemma_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 6,
        gradient_accumulation_steps = 4,    # Use GA to mimic batch size!
        output_dir = "output_checkpoint",   #Necessary, if you want to save checkpoint and later continue
        save_strategy = "steps",            #Necessary, if you want to save checkpoint and later continue
        save_steps = 25,                     #Necessary, if you want to save checkpoint and later continue
        save_total_limit=3,                 #Only save latest 2 checkpoints!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/176236 [00:00<?, ? examples/s]

In [12]:
# Next piece of training!
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=96):   0%|          | 0/176236 [00:00<?, ? examples/s]

In [19]:
# Id recommend using the refined dataset instead, config changes cause us issues later!
"""
def is_valid(example):
    try:
        _ = tokenizer(example["text"])
        return True
    except Exception:
        return False

filtered_dataset = dataset.filter(is_valid, num_proc=2)
"""

# ignore_index is a pyhton pandas command i believe, it causes issues in model and stops execution, so we have to add this piece or filte rout all training data with it
if not hasattr(model.config, "ignore_index"):
    model.config.ignore_index = -100

In [13]:
# unzip is installed in colab, not  in runpod, so have to do if running on runpod!
!apt-get update && apt-get install -y unzip

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1765 kB]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4476 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/

In [15]:
# unzip the output checkpoints!
!unzip "output_checkpoint"

Archive:  output_checkpoint.zip
  inflating: output_checkpoint/checkpoint-7343/trainer_state.json  
  inflating: output_checkpoint/checkpoint-7343/rng_state.pth  
  inflating: output_checkpoint/checkpoint-7343/scheduler.pt  
  inflating: output_checkpoint/checkpoint-7343/optimizer.pt  
  inflating: output_checkpoint/checkpoint-7343/training_args.bin  
  inflating: output_checkpoint/checkpoint-7343/processor_config.json  
  inflating: output_checkpoint/checkpoint-7343/chat_template.json  
  inflating: output_checkpoint/checkpoint-7343/tokenizer.json  
  inflating: output_checkpoint/checkpoint-7343/tokenizer.model  
  inflating: output_checkpoint/checkpoint-7343/added_tokens.json  
  inflating: output_checkpoint/checkpoint-7343/special_tokens_map.json  
  inflating: output_checkpoint/checkpoint-7343/tokenizer_config.json  
  inflating: output_checkpoint/checkpoint-7343/preprocessor_config.json  
  inflating: output_checkpoint/checkpoint-7343/adapter_config.json  
  inflating: output_chec

In [ ]:
# To start training, checkpoints will start auto compiling
trainer_stats = trainer.train()

In [16]:
# # For resuming if stoped midway
# trainer_stats = trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 176,236 | Num Epochs = 1 | Total steps = 7,343
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 4 x 1) = 24
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Step,Training Loss


In [ ]:
trainer_stats

In [18]:
# Saving model LORA weights only, not the full model!
model.save_pretrained("gemma-3coder_loraweights")  # Local saving
tokenizer.save_pretrained("gemma-3coder_loraweights")

['gemma-3coder_loraweights/processor_config.json']

In [19]:
# Push to Hugging Face (private repo), only the LORA weights
trainer.model.push_to_hub("hiteshmodi/unsloth_gemma-3coder_loraweights", private=True)
trainer.tokenizer.push_to_hub("hiteshmodi/unsloth_gemma-3coder_loraweights", private=True)

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Saved model to https://huggingface.co/hiteshmodi/unsloth_gemma-3coder_loraweights


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

In [ ]:
# Now, we will convert the model into binary that Ollama can use without requiring pytorch or transformers, its gguf format.

In [21]:
# trying to use solution we got from github issues section for unloading full model, use model merge then save pretrained.
# I used an additional save_pretrained_merged function call as a workaround. It creates the config.json correctly, and the subsequent gguf creation goes through.

model.save_pretrained_merged("gemma-3-finetune_coder", tokenizer)

model.save_pretrained_gguf(
"gemma-3-finetune_coder",
quantization_type = "Q8_0",
)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [00:38<00:38, 38.07s/it]

...

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [01:04<00:00, 32.08s/it]


Unsloth: Updating system package directories
Unsloth: Install GGUF and other packages
Unsloth GGUF:hf-to-gguf:Loading model: gemma-3-finetune_coder
Unsloth GGUF:hf-to-gguf:Model architecture: Gemma3ForConditionalGeneration
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> Q8_0, shape = {2560, 262208}
Unsloth GGUF:hf-to-gguf:gguf: loading model part 'model-00002-of-00002.safetensors'
Unsloth GGUF:hf-to-gguf:output_norm.weight,                torch.bfloat16 --> F32, shape = {2560}
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GG

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth: Converted to gemma-3-finetune_coder.Q8_0.gguf with size = 4.1G
Unsloth: Successfully saved GGUF to:
gemma-3-finetune_coder.Q8_0.gguf


['gemma-3-finetune_coder.Q8_0.gguf']

In [22]:
# its a 3.9G file, this is the full model that can be used with Ollama now
ls -lh gemma-3-finetune_coder.Q8_0.gguf

-rw-rw-rw- 1 root root 3.9G Jun  8 06:23 gemma-3-finetune_coder.Q8_0.gguf


In [26]:
# Push to Hugging Face (private repo), only the LORA weights
trainer.model.push_to_hub("hiteshmodi/unsloth_gemma3coderfull-4b-it", private=True)
trainer.tokenizer.push_to_hub("hiteshmodi/unsloth_gemma3coderfull-4b-it", private=True)

README.md:   0%|          | 0.00/601 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Saved model to https://huggingface.co/hiteshmodi/unsloth_gemma3coderfull-4b-it


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

In [23]:
# moules required for uploading thi sto huggingface
from huggingface_hub import HfApi

In [27]:
api = HfApi()
api.upload_file(
    path_or_fileobj="gemma-3-finetune_coder.Q8_0.gguf",
    path_in_repo="gemma-3-finetune_coder.Q8_0.gguf",
    repo_id="hiteshmodi/unsloth_gemma3coderfull-4b-it",
    repo_type="model"
)

  0%|          | 0/1 [00:00<?, ?it/s]

gemma-3-finetune_coder.Q8_0.gguf:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hiteshmodi/unsloth_gemma3coderfull-4b-it/commit/dc71196d905d7aa5849590116b1022d56fa0e618', commit_message='Upload gemma-3-finetune_coder.Q8_0.gguf with huggingface_hub', commit_description='', oid='dc71196d905d7aa5849590116b1022d56fa0e618', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hiteshmodi/unsloth_gemma3coderfull-4b-it', endpoint='https://huggingface.co', repo_type='model', repo_id='hiteshmodi/unsloth_gemma3coderfull-4b-it'), pr_revision=None, pr_num=None)

In [30]:
# Lets upload the output checkpoints as well!
api.upload_file(
    path_or_fileobj="output_checkpoint.zip",
    path_in_repo="output_checkpoint.zip",
    repo_id="hiteshmodi/unsloth_gemma3coderfull-4b-it",
    repo_type="model"
)

  0%|          | 0/1 [00:00<?, ?it/s]

output_checkpoint.zip:   0%|          | 0.00/274M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hiteshmodi/unsloth_gemma3coderfull-4b-it/commit/92f220ba213da70db9789c99555006364352fb3e', commit_message='Upload output_checkpoint.zip with huggingface_hub', commit_description='', oid='92f220ba213da70db9789c99555006364352fb3e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hiteshmodi/unsloth_gemma3coderfull-4b-it', endpoint='https://huggingface.co', repo_type='model', repo_id='hiteshmodi/unsloth_gemma3coderfull-4b-it'), pr_revision=None, pr_num=None)